In [1]:
from tqdm import tqdm
import csv
import numpy as np

UMLS_KEY = "43f9234c-4977-45f6-a440-2dda1b43d919"

In [2]:
# get wumls and english names
# encode all english names and build index
# encode query and find nearest neighbour from index

### load wumls and english names

In [3]:
wumls_file = '/home/tim/MedicalLay/WUMLS/MRCONSO_WUMLS_GER.RRF'

cuis = set()
with open(wumls_file, newline='\n') as csvfile:
    spamreader = csv.reader(csvfile, delimiter='|')
    for row in spamreader:
        cui = row[0]
        cuis.add(cui)

print(len(cuis))

110121


In [4]:
import sqlite3

db_path = "/home/tim/PycharmProjects/py-umls/databases/umls.db"
db = sqlite3.connect(db_path)
cur = db.cursor()

In [5]:
for res in cur.execute("SELECT * FROM descriptions WHERE LAT = 'ENG' LIMIT 5"):
    print(res)

('C0000005', 'ENG', 'MSH', 'PEP', '(131)I-Macroaggregated Albumin', 'T116|T121|T130')
('C0000039', 'ENG', 'MTH', 'PN', '1,2-dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000039', 'ENG', 'MSH', 'MH', '1,2-Dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000039', 'ENG', 'MSH', 'PM', '1,2 Dipalmitoylphosphatidylcholine', 'T109|T121')
('C0000052', 'ENG', 'MTH', 'PN', '1,4-alpha-Glucan Branching Enzyme', 'T116|T126')


In [6]:
cui = "C0005903"
def get_prefered_name_of_cui(cuis):
    cui_names_mapping = {}
    not_found = 0
    for identifier in tqdm(cuis):
        sql = f"SELECT STR FROM descriptions WHERE CUI = ? ORDER BY CASE WHEN SAB = 'MTH' THEN 1 WHEN SAB = 'SNOMEDCT' THEN 2  WHEN SAB = 'MSH' THEN 3 ELSE 4 END LIMIT 1"
        
        try:
            res = next(cur.execute(sql, (identifier,)))[0]
            cui_names_mapping[identifier] = res
        except StopIteration:
            # print('Search term ' + "'" + str(identifier) + "'" + ' not found')
            not_found += 1
    print(f"{not_found} not found from {len(cuis)}")
    return cui_names_mapping

cui_names_mapping = get_prefered_name_of_cui(cuis)

100%|██████████| 110121/110121 [00:03<00:00, 29777.83it/s]

574 not found from 110121


In [7]:
concept_cuis, concept_names = zip(*cui_names_mapping.items())

In [11]:
with open("english_wumls.csv","w", newline="") as f:
    spamwriter = csv.writer(f, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for cui, name in cui_names_mapping.items():
        spamwriter.writerow([cui, name])

### load sapbert

In [8]:
from transformers import AutoTokenizer, AutoModel  
tokenizer = AutoTokenizer.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext")  
model = AutoModel.from_pretrained("cambridgeltl/SapBERT-from-PubMedBERT-fulltext") #.cuda(1)

#### encode umls labels

In [9]:
bs = 128
all_reps = []
for i in tqdm(np.arange(0, len(concept_names), bs)):
    toks = tokenizer.batch_encode_plus(concept_names[i:i+bs], 
                                       padding="max_length", 
                                       max_length=25, 
                                       truncation=True,
                                       return_tensors="pt")
    #toks_cuda = {}
    #for k,v in toks.items():
    #    toks_cuda[k] = v.cuda(1)
    #output = model(**toks_cuda)
    
    output = model(**toks)
    cls_rep = output[0][:,0,:]
    
    all_reps.append(cls_rep.cpu().detach().numpy())
all_reps_emb = np.concatenate(all_reps, axis=0)

  4%|███████▍                                                                                                                                                                                          | 33/856 [01:54<47:36,  3.47s/it]


KeyboardInterrupt: 

In [18]:
print(all_reps_emb.shape)

NameError: name 'all_reps_emb' is not defined

NameError: name 'all_reps_emb' is not defined

NameError: name 'all_reps_emb' is not defined

NameError: name 'all_reps_emb' is not defined

NameError: name 'all_reps_emb' is not defined

NameError: name 'all_reps_emb' is not defined

#### encode query

In [19]:
query = "cardiopathy"
query_toks = tokenizer.batch_encode_plus([query], 
                                       padding="max_length", 
                                       max_length=25, 
                                       truncation=True,
                                       return_tensors="pt")

In [20]:
query_output = model(**query_toks)
query_cls_rep = query_output[0][:,0,:]

In [21]:
query_cls_rep.shape

torch.Size([1, 768])

#### find query's nearest neighbour

In [22]:
# for large-scale search, should switch to faiss
from scipy.spatial.distance import cdist

In [23]:
dist = cdist(query_cls_rep.cpu().detach().numpy(), all_reps_emb)
nn_index = np.argmin(dist)
print ("predicted label:", snomed_sf_id_pairs_100k[nn_index])

NameError: name 'all_reps_emb' is not defined